# Import statements

In [1]:
# Cell 1: Import Libraries
import geopandas as gpd
import pandas as pd
import glob
import os
import json
import plotly.express as px
import dash
from dash import dcc, html, Input, Output, Dash
# Set SHAPE_RESTORE_SHX environment variable
os.environ["SHAPE_RESTORE_SHX"] = "YES"

In [2]:
# Read the JSON file
def read_json(filename):
    with open(filename, 'r') as file:
        gemmingen_data = json.load(file)
    return gemmingen_data

In [3]:
def open_gdf_file(file_path):
    gdf = gpd.read_file(file_path)
    return gdf

In [5]:
# Cell 2: Define Root Folder and Initialize List
root_folder = "data/Erntejahr 2024 (Weizen, komplett)"

calculate_emissions_json = read_json("gemmingen.json")
gemmingen_area = calculate_emissions_json["area"]

# Initialize an empty list to store shapefile paths
shapefile_list = []

# Traverse the directory structure
for subdir, _, _ in os.walk(root_folder):
    # Look for .shp files in the 'doc' folder of each subdirectory
    shapefiles = glob.glob(os.path.join(subdir, "doc", "*.shp"))
    shapefile_list.extend(shapefiles)


# Cell 3: Read Shapefiles and Combine into a Single GeoDataFrame
gdfs = []
for shapefile_path in shapefile_list:
    gdf = open_gdf_file(shapefile_path)
    # Append the GeoDataFrame to a list
    gdfs.append(gdf)

# Concatenate all GeoDataFrames into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True), crs=gdfs[0].crs)

# Fill missing values with NA
combined_gdf = combined_gdf.fillna(0)

# Cell 4: Display Output and Columns
print("All shapefiles combined successfully!")

All shapefiles combined successfully!


In [6]:
def getValues(column):
    percent = calculate_emissions_json[column]["Zusammensetzung"]
    emissionfactor = calculate_emissions_json[column]["Emissionsfaktor"]
    return percent, emissionfactor

In [7]:
# combined_gdf['Product'] = combined_gdf['Product'].apply(lambda x: x.split()[0])

# Cell 6: Calculate Emissions for Each Product
def calculate_emissions(row):
    product = row["Product"]
    appliedRate = row["AppliedRate"]
    if product in calculate_emissions_json:
        if calculate_emissions_json[product]["Zusammensetzung"]:
            if isinstance(calculate_emissions_json[product]["Zusammensetzung"], dict):
                composition = calculate_emissions_json[product]["Zusammensetzung"]["N"]
            else:
                composition = calculate_emissions_json[product]["Zusammensetzung"]
            emissions_factor = calculate_emissions_json[product]["Emissionsfaktor"]
            emissions = composition * appliedRate * emissions_factor
        else:
            emissions = appliedRate * calculate_emissions_json[product]["Emissionsfaktor"]
    else:
        emissions = 0
    return emissions

combined_gdf["Emissions"] = combined_gdf.apply(calculate_emissions, axis=1)
combined_gdf["FuelEmissions"] = combined_gdf["FUEL"] * calculate_emissions_json["FUEL"]["Emissionsfaktor"]
if "VRYIELDMAS" in combined_gdf.columns and "DRYMATTER" in combined_gdf.columns:
    combined_gdf["YieldEmissions"] = combined_gdf["VRYIELDMAS"] * combined_gdf["DRYMATTER"] * calculate_emissions_json["yield"]["Zusammensetzung"] * calculate_emissions_json["yield"]["Emissionsfaktor"]
else:
    combined_gdf["YieldEmissions"] = 0
# combined_gdf["YieldEmissions"] = combined_gdf["VRYIELDMAS"] *  combined_gdf["DRYMATTER"] * calculate_emissions_json["yield"]["Zusammensetzung"] * calculate_emissions_json["yield"]["Emissionsfaktor"]

In [8]:
average_emissions = combined_gdf['Emissions'].mean() + combined_gdf['FuelEmissions'].mean() + combined_gdf['YieldEmissions'].mean()
print(average_emissions * gemmingen_area)

2115873.357712619


In [9]:
combined_gdf_grouped = combined_gdf.groupby("Product").agg({"Emissions": "sum"}) * gemmingen_area
combined_gdf_grouped

,Emissions
Product,
0,0.000000e+00
ASS 26N,5.794823e+08
Broadway Plus,3.230304e+06
Curbatur + Caramba + Karate,6.705583e+06
Elatus Era + Countdown+U46M,1.201258e+07
Kalkamonsalpeter 27,6.729441e+08
Menge,1.315322e+10
Mertil + Herbosol,1.642850e+06


In [10]:
# Cell 5: Save Combined GeoDataFrame as a Shapefile
output_shapefile_path = os.path.join(root_folder, "combined_data.geojson")
combined_gdf.to_file(output_shapefile_path,driver="GeoJSON")

In [ ]:
def gen_dash_plot(gdf, columnName, title):
    # Convert the GeoDataFrame geometry to latitude and longitude
    gdf["lon"] = gdf.geometry.centroid.x
    gdf["lat"] = gdf.geometry.centroid.y

     # Get the minimum and maximum values of the column
    min_val = gdf[columnName].min()
    max_val = gdf[columnName].max()

    scatter_fig = px.scatter_mapbox(
        gdf,
        lat="lat",
        lon="lon",
        color=columnName,
        hover_name=columnName,
        hover_data={"lon": ":.5f", "lat": ":.5f"},  # Display coordinates with 5 decimal precision
        title=title,
        mapbox_style="open-street-map",
        zoom=15,
        height=600,
        range_color=[min_val, max_val],
    )
    scatter_fig.update_traces(marker=dict(size=10, opacity=0.7))  # Customize marker size and opacity
    return scatter_fig


In [ ]:
# Create a Dash app
app = Dash(__name__)

# Sample GeoDataFrame (replace with your actual data)
# gdf = gpd.read_file("path_to_your_shapefile.shp")
columns = ["FUEL", "AppliedRate", "Emissions", "FuelEmissions", "YieldEmissions"]  # Non-geometry columns

# App layout
app.layout = html.Div([
    html.H1("Interactive Scatter Mapbox", style={'text-align': 'center'}),

    # Dropdown for selecting the column to color by
    html.Div([
        html.Label("Select Column to Visualize:"),
        dcc.Dropdown(
            id="color-dropdown",
            options=[{"label": col, "value": col} for col in columns],
            value=columns[0],  # Default value
            clearable=False,
        )
    ], style={'width': '50%', 'margin': 'auto'}),

    # Map visualization
    dcc.Graph(id="mapbox-plot"),
])

# Callback to update the scatter plot dynamically
@app.callback(
    Output("mapbox-plot", "figure"),
    Input("color-dropdown", "value")
)
def update_mapbox(selected_column):
    return gen_dash_plot(combined_gdf, selected_column, title=f"Scatter Mapbox - {selected_column}")

# Run the app
if __name__ == "__main__":
    app.run_server(debug=True)


FertilizerAAS

Kalkammonsalpeter
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/carbonEmission/$value

Harnstoff
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTIwNV8xMjIwXzIxNDJfNzAzMw==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTIwNV8xMjIwXzIxNDJfNzAzMw==/submodel-elements/carbonEmission/$value

Ammoniumsulfatsalpeter
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMjEzNV8xMjIwXzIxNDJfMjI5OQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMjEzNV8xMjIwXzIxNDJfMjI5OQ==/submodel-elements/carbonEmission/$value

Diammonphosphat
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzI3NV8xMjIwXzIxNDJfOTMyNQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzI3NV8xMjIwXzIxNDJfOTMyNQ==/submodel-elements/carbonEmission/$value

Gülle
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTExMF8yMjIwXzIxNDJfODI4NQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTExMF8yMjIwXzIxNDJfODI4NQ==/submodel-elements/carbonEmission/$value


FuelAAS

dataAttributes
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzQ2M18yMjIwXzIxNDJfNTcwMA==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzQ2M18yMjIwXzIxNDJfNTcwMA==/submodel-elements/carbonEmission/$value

YieldAAS

dataAttributes
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzA3Ml8yMjIwXzIxNDJfOTgxNw==/submodel-elements/yieldMass/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzA3Ml8yMjIwXzIxNDJfOTgxNw==/submodel-elements/dryMatter/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzA3Ml8yMjIwXzIxNDJfOTgxNw==/submodel-elements/carbonEmission/$value

CropProtection

GeneralPesticides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDMxNF8xMDMwXzIxNDJfNDA0OA==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDMxNF8xMDMwXzIxNDJfNDA0OA==/submodel-elements/carbonEmission/$value

Fungicides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTEzNF8xMDMwXzIxNDJfNTk2NQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMTEzNF8xMDMwXzIxNDJfNTk2NQ==/submodel-elements/carbonEmission/$value

Herbicides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzMzNF8xMDMwXzIxNDJfMzQxMQ==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vMzMzNF8xMDMwXzIxNDJfMzQxMQ==/submodel-elements/carbonEmission/$value

Insecticides
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzUzNF8xMDMwXzIxNDJfMTg2OA==/submodel-elements/amount/$value
http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNzUzNF8xMDMwXzIxNDJfMTg2OA==/submodel-elements/carbonEmission/$value




In [ ]:
import requests
import time
# Define the URL for the PATCH request
value_url = "http://192.168.0.178:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value"
emission_url = "http://localhost:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/carbonEmission/$value"

# Define the headers, if required
headers = {
    "Content-Type": "application/json",  # Adjust based on the API's requirement
}

# Define the string to send in the body
body = "This is a string to send in the PATCH request"

# Send the PATCH request

# Read the combined_gdf product field and check if it contains the Kalkammonsalpeter and read the AppliedRate as str
if "Kalkamonsalpeter 27" in combined_gdf["Product"].values:
    applied_rate_str = combined_gdf.loc[combined_gdf["Product"] == "Kalkamonsalpeter 27", "AppliedRate"].astype(str).values
    # body = json.dumps({"value": applied_rate_str})

    for value in applied_rate_str:
        response = requests.patch(value_url, json=value, headers=headers)
        # Check the status code
        if response.status_code == 204:
            print("PATCH request successful!")
            print("Response:", response.text)
        else:
            print(f"Failed with status code {response.status_code}: {response.text}")
#     response = requests.patch(value_url, data=body, headers=headers)
#     # Check the status code
#     if response.status_code == 200:
#         print("PATCH request successful!")
#         print("Response:", response.text)
#     else:
#         print(f"Failed with status code {response.status_code}: {response.text}")


In [23]:
import requests
import json

url = "http://192.168.0.178:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value"
payload = "121"
response = requests.patch(url, json=payload, headers=headers)
try:
    print(f"Sent PATCH to {url} - Status Code: {response.status_code}, Response: {response.json()}")
except json.JSONDecodeError:
    print(f"Sent PATCH to {url} - Status Code: {response.status_code}, Response Text: {response.text}")

Sent PATCH to http://192.168.0.178:8081/submodels/aHR0cHM6Ly9leGFtcGxlLmNvbS9pZHMvc20vNDQ2NF8xMjIwXzIxNDJfNDQ4Mg==/submodel-elements/amount/$value - Status Code: 204, Response Text: 
